In [19]:
import datetime
import os
import sys
import numpy as np
import pandas as pd
from osgeo import gdal
import xarray as xr

from pathlib import Path

from tqdm import tqdm

from joblib import Parallel, delayed

## GSMaP monthly dat to netCDF

##### funcs

In [20]:
def read_dat(dat_file, data_shape=None):
    # monthly数据分为两层 
    # 包含两个全球字段：月平均降雨率；以及每月有效像素（≧0毫米）的数量。前者的单位是[毫米/小时]，缺失值为-999.9。两层数据相乘得到的是月总降水量[毫米/月]。
    
    # with open(dat_file, 'rb'):
    dat_data = np.fromfile(dat_file, dtype='<f', count=-1)

    # reshape
    if data_shape:
        dat_data = dat_data.reshape(data_shape)
        

    return dat_data


def to_1month_netcdf(data, time, geotransform, nc_var_name):
    
    # # expand dims for 1day
    # # get data shape
    # if len(data.shape) == 2:
    #     lat_len, lon_len = data.shape
    #     data = np.expand_dims(data, axis=0)
    # else:
    #     _, lat_len, lon_len = data.shape
    
    # get data shape
    lat_len, lon_len = data.shape
    
    # calc lon and lat series
    lon = np.arange(geotransform[0], geotransform[0] + geotransform[1] * lon_len, geotransform[1])
    lon = np.round(lon, 2)
    lat = np.arange(geotransform[3], geotransform[3] + geotransform[5] * lat_len, geotransform[5])
    lat = np.round(lat, 2)

    
    # create xarray dataset
    dst = xr.Dataset(
        {
            'precipitation': (['latitude', 'longitude'], data)
        },
        coords={
            'longitude': lon,
            'latitude': lat,
            'time': time
        }
    )
    
    dst.attrs['title'] = nc_var_name + 'Monthly Precipitation'
    dst.attrs['description'] = f'GSMap {nc_var_name} Monthly Precipitation'
    
    dst['precipitation'].attrs['units'] = 'mm/month'
    dst['precipitation'].attrs['long_name'] = 'Monthly Precipitation'
    
    return dst


##### run

In [21]:
glob_gsmap_geotrans = (0, 0.1, 0, 60, 0, -0.1)
glob_varname = 'GSMaP-STD-V8-G'
glob_datdir = Path(r'D:\DATA\gsmap_testdata\monthly_test\input')
glob_ncdir = Path(r'D:\DATA\gsmap_testdata\monthly_test\output_nc')


In [22]:
# 用于文件名切片
st, ed = 12, 18

In [23]:
dat_files = list(glob_datdir.glob('*.dat'))
# 修改日期字符串 格式如 20190101
test_dt = dat_files[0].name[st:ed]
print(test_dt)
datetime.datetime.strptime(test_dt, '%Y%m')

202001


datetime.datetime(2020, 1, 1, 0, 0)

In [24]:
for d in dat_files:
    print(f'[ INFO ] Convert - {d.stem}')
    # read dat
    data = read_dat(d, data_shape=(2, 1200, 3600))
    # convert to mm/month
    rain_rate = data[0]
    rain_rate[rain_rate < 0] = 0
    rain_count = data[1]
    
    data = rain_rate * rain_count

    # create time
    dt_str = d.name[st:ed]
    
    
    time = pd.to_datetime(dt_str, format='%Y%m')
    date_ = pd.date_range(time, periods=1 , freq='D')[0]
    
    # to netcdf
    nc_var_name = glob_varname
    dst = to_1month_netcdf(data, date_, glob_gsmap_geotrans, nc_var_name)
    
    # save
    nc_file = glob_ncdir.joinpath(f'{nc_var_name}_{dt_str}.nc')
    dst.to_netcdf(nc_file)

[ INFO ] Convert - gsmap_gauge.202001.0.1d.monthly.v8.0000.1
